In [9]:
import os
import sys
import pandas as pd
import re

In [2]:
sys.path.append('F:\Frederik\pfizer_mcrpc\paqc')

In [3]:
from paqc.report import report
from paqc.connectors import csv
from paqc.driver import driver
from paqc.utils import config_utils

In [75]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# Make a subset of around 700 to be able to test things
df = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\total.csv')
df = df.iloc[::100, :]
df.to_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\subset_1000.csv', index=False)

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,9,20,21,24,25,28,29,40,41,44,45,48,49,64,65,76,77,80,81,92,93,100,101,108,109,120,121,124,125,128,129,136,137,164,165,184,185,188,189,192,193,200,201,204,205,220,221,236,237,252,253,256,257,272,273,276,277,284,285,304,305,308,309,316,317,324,325,328,329,332,333,340,341,348,349,352,353,368,369,388,389,396,397,404,405,452,453,460,461,468,469,492,493,524,525,532,533,536,537,540,541,544,545,548,549,556,557,592,593,600,601,628,629,640,641,652,653,664,665,668,669,672,673,680,681,712,713,716,717,720,721,748,749,752,753,768,769,792,793,800,801,816,817,820,821,824,825,828,829,844,845,852,853,896,897,912,913,920,921,940,941,944,945,948,949,956,957,964,965,988,989,1000,1001,1012,1013,1028,1029,1040,1041,1060,1061,1064,1065,1068,1069,1076,1077,1080,1081,1084,1085,1128,1129,1180,1181,1204,1205,1208,1209,1212,1213,1244,1245,1260,1261,1280,1281,1284,1285,1304,1305,1312,1313,1344,1345,1408,1409

In [8]:
dict_config = config_utils.config_open(r'F:\Frederik\pfizer_mcrpc\pfizer_mcrpc\raw_data\QC\general_config.yml')[1]

<h1> First QC run </h1>

In [8]:
# Viewing from the htmls, we can go through the qcs that didn't pass, qc19:
idx_wrong = [794, 9195, 13271, 24971, 55398, 57644, 68542]
df_index = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\total.csv', usecols=['INDEX_DATE'])

In [11]:
# Data is fine, qc19 changed so that it's bigger or equal, instead of bigger
df_index.loc[idx_wrong]

,INDEX_DATE
794,01/01/2014
9195,01/01/2014
13271,01/01/2014
24971,01/01/2014
55398,01/01/2014
57644,01/01/2014
68542,01/01/2014


In [21]:
# qc1, some column names have trailing spaces
faulty_colnames = pd.read_csv('report/extra3_2017-12-11_18_32_10.csv', header=None)[0].tolist()
improved_colnames = [faulty_colname.strip() for faulty_colname in faulty_colnames]
mapping_colnames = {old: new for old, new in zip(faulty_colnames, improved_colnames)}

In [29]:
df = df.rename(columns = mapping_colnames)

In [30]:
# qc15, PN_FLAG is the label and has values N and P, but is expected to be numeric, so problem
df['PN_FLAG'] = df['PN_FLAG'].map({'N':0, 'P': 1})

In [36]:
# Export to CSV, every of these QCs should pass now
df.to_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\total.csv', index=False)

<h1> Second QC run </h1>

In [125]:
# Run the qcs again, with some extras
d = driver.Driver(r'F:\Frederik\pfizer_mcrpc\pfizer_mcrpc\raw_data\QC\general_config.yml')
d.run()

Loading config file...
Checking and parsing config file...
Config file checked and parsed. Starting QC pipeline...
-------------------------------------------------------
Starting QCs on input1, file path: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc9 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc9 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc1 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc3 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc4 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc6 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc7 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc8 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc11 on input1: F:/Projects/Pfizer_mCRPC/Data/Raw_data/total.csv
Executing test qc12 on input1: F:

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
df = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\total.csv')

In [5]:
# Most columns have dates on the index, never after. But on is still a problem
late_cols = pd.read_csv('report/extra2_2017-12-11_18_32_10.csv')
late_cols = late_cols.values.tolist()
late_cols = [late_col[0] for late_col in late_cols]
late_cols.insert(0, 'INDEX_DATE')

In [6]:
df_prob =  df[late_cols].apply(lambda x: pd.to_datetime(x, format = '%m/%d/%Y'))

In [7]:
# To give an idea of how many rows are not null for each feature
ss_notnull = (~df_prob.isnull()).sum()

In [8]:
# No value is after it's index date
df_prob.apply(lambda x: x > df_prob['INDEX_DATE']).sum().sum()

0

In [9]:
# But plenty are on the index date
ss_onindex = df_prob.apply(lambda x: x >= df_prob['INDEX_DATE']).sum()

In [15]:
df_onindex = pd.DataFrame({'OnIndex':(ss_onindex/ss_notnull), 'NotNull': ss_notnull})

<h1> Check if X/Z prescription is strictly after index date </h1>

In [91]:
df_flatfile = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\PFA\Pfizer_mCRPC_IDP_v2.csv')

In [92]:
x_z_colnames = ['ENZALUTAMIDE_1ST_RX_DT_OVERALL', 'ENZALUTAMIDE_FIRST_RX_DT_POST', 
                'ABIRATERONE_1ST_RX_DT_OVERALL', 'ABIRATERONE_FIRST_RX_DT_POST']

In [93]:
df_xz = df_flatfile[x_z_colnames]

In [94]:
df_xz=  df_xz.apply(lambda x: pd.to_datetime(x, format = '%m/%d/%Y'))

In [95]:
df_xz['PATIENT_ID'] = df_flatfile['PATIENT_ID']

In [96]:
df_xz.head()

,ENZALUTAMIDE_1ST_RX_DT_OVERALL,ENZALUTAMIDE_FIRST_RX_DT_POST,ABIRATERONE_1ST_RX_DT_OVERALL,ABIRATERONE_FIRST_RX_DT_POST,PATIENT_ID
0,NaT,NaT,NaT,NaT,52378448
1,NaT,NaT,NaT,NaT,268555353
2,NaT,NaT,NaT,NaT,61671419
3,NaT,NaT,NaT,NaT,1320563705
4,NaT,NaT,NaT,NaT,861053886


In [97]:
df_index = pd.read_excel(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Pfizer_mCRPC_Patients.xlsx')
df_index = df_index[['PATIENT_ID', 'PN_FLAG', 'INDEX_DATE']]

In [98]:
df_index['INDEX_DATE'] = pd.to_datetime(df_index['INDEX_DATE'], format = '%m/%d/%Y')

In [99]:
df_index.head()

,PATIENT_ID,PN_FLAG,INDEX_DATE
0,377317624,N,2016-04-21
1,73066609,N,2017-05-08
2,70175556,N,2016-06-20
3,713474282,N,2016-04-24
4,1563563420,N,2014-05-14


In [100]:
df = df_index.merge(df_xz, on='PATIENT_ID', how='left')

In [101]:
df.head()

,PATIENT_ID,PN_FLAG,INDEX_DATE,ENZALUTAMIDE_1ST_RX_DT_OVERALL,ENZALUTAMIDE_FIRST_RX_DT_POST,ABIRATERONE_1ST_RX_DT_OVERALL,ABIRATERONE_FIRST_RX_DT_POST
0,377317624,N,2016-04-21,NaT,NaT,NaT,NaT
1,73066609,N,2017-05-08,NaT,NaT,NaT,NaT
2,70175556,N,2016-06-20,NaT,NaT,NaT,NaT
3,713474282,N,2016-04-24,NaT,NaT,NaT,NaT
4,1563563420,N,2014-05-14,2016-07-18,2016-07-18,NaT,NaT


In [102]:
# No patient ever had X or Z before index date
df[x_z_colnames].apply(lambda x: x < df['INDEX_DATE']).sum()

ENZALUTAMIDE_1ST_RX_DT_OVERALL    0
ENZALUTAMIDE_FIRST_RX_DT_POST     0
ABIRATERONE_1ST_RX_DT_OVERALL     0
ABIRATERONE_FIRST_RX_DT_POST      0
dtype: int64

In [103]:
# A (small) amount of patients was prescribed on the index date
df[x_z_colnames].apply(lambda x: x == df['INDEX_DATE']).sum()

ENZALUTAMIDE_1ST_RX_DT_OVERALL    59
ENZALUTAMIDE_FIRST_RX_DT_POST     59
ABIRATERONE_1ST_RX_DT_OVERALL     80
ABIRATERONE_FIRST_RX_DT_POST      80
dtype: int64

In [104]:
# All of which belong to the positive class
df.loc[df['PN_FLAG'] == 'P', x_z_colnames].apply(lambda x: x == df.loc[df['PN_FLAG'] == 'P', 'INDEX_DATE']).sum()

ENZALUTAMIDE_1ST_RX_DT_OVERALL    59
ENZALUTAMIDE_FIRST_RX_DT_POST     59
ABIRATERONE_1ST_RX_DT_OVERALL     80
ABIRATERONE_FIRST_RX_DT_POST      80
dtype: int64

In [105]:
# Check how many in total (X or Z)
df[x_z_colnames].apply(lambda x: x == df['INDEX_DATE']).any(axis=1).sum()

139

In [108]:
# List those patients
df_xz_on_index = df[df[x_z_colnames].apply(lambda x: x == df['INDEX_DATE']).any(axis=1)]

In [110]:
df_xz_on_index.to_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Patients_XZ_on_index.csv', index=False)

<h1> Check how many of patients with X/Z on index date are in the EMR data </h1>

In [111]:
df_xz_on_index = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Patients_XZ_on_index.csv')

In [113]:
df_onc_emr = pd.read_excel(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\patients_link_OncEMR.xlsx')

In [121]:
pat_id_xz_on_index = set(df_xz_on_index['PATIENT_ID'].tolist())

In [124]:
# Number of the 139 patients with X/Z on index date that are also in the oncology EMR dataset
df_onc_emr['PATIENT_ID'].isin(pat_id_xz_on_index).sum()

10

<h1> Check if x or z is within 90 days for positive cohort </h1>

In [54]:
df_pos = df[df['PN_FLAG'] == 'P']
df_neg = df[df['PN_FLAG'] == 'N']

In [59]:
(df_pos[x_z_colnames].min(axis = 1) - df_pos['INDEX_DATE']).max()

Timedelta('90 days 00:00:00')

In [61]:
(df_neg[x_z_colnames].min(axis = 1) - df_neg['INDEX_DATE']).min()

Timedelta('91 days 00:00:00')

<h1> Check number of each group of predictors </h1>

In [33]:
df_total = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\total.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8,9,20,21,24,25,28,29,40,41,44,45,48,49,64,65,76,77,80,81,92,93,100,101,108,109,120,121,124,125,128,129,136,137,164,165,184,185,188,189,192,193,200,201,204,205,220,221,236,237,252,253,256,257,272,273,276,277,284,285,304,305,308,309,316,317,324,325,328,329,332,333,340,341,348,349,352,353,368,369,388,389,396,397,404,405,452,453,460,461,468,469,492,493,524,525,532,533,536,537,540,541,544,545,548,549,556,557,592,593,600,601,628,629,640,641,652,653,664,665,668,669,672,673,680,681,712,713,716,717,720,721,748,749,752,753,768,769,792,793,800,801,816,817,820,821,824,825,828,829,844,845,852,853,896,897,912,913,920,921,940,941,944,945,948,949,956,957,964,965,988,989,1000,1001,1012,1013,1028,1029,1040,1041,1060,1061,1064,1065,1068,1069,1076,1077,1080,1081,1084,1085,1128,1129,1180,1181,1204,1205,1208,1209,1212,1213,1244,1245,1260,1261,1280,1281,1284,1285,1304,1305,1312,1313,1344,1345,1408,1409,1

In [57]:
colnames = df_total.columns
set_prefixes = set([colname.split('_')[0] for colname in colnames])

In [71]:
# Number of CPT predictors
prog = re.compile(r'^C')
cpt = [prog.match(prefix).string for prefix in set_prefixes if prog.match(prefix)]

# Number of GPI-6 predictors
prog = re.compile(r'^G')
gpi = [prog.match(prefix).string for prefix in set_prefixes if prog.match(prefix)]

# Number of ICD predictors
prog = re.compile(r'^D')
icd = [prog.match(prefix).string for prefix in set_prefixes if prog.match(prefix)]

# Number of HCPCS predictors
prog = re.compile(r'^H')
hcpcs = [prog.match(prefix).string for prefix in set_prefixes if prog.match(prefix)]

# Number of Specialty predictors
prog = re.compile(r'^S')
specialty = [prog.match(prefix).string for prefix in set_prefixes if prog.match(prefix)]

In [72]:
# Total is not 886
len(cpt + gpi + icd + hcpcs + specialty)

882

In [76]:
len(icd)
len(cpt)
len(hcpcs)
len(gpi)
len(specialty)

277

304

93

145

63

In [77]:
# Check the raw cpt data, as it's the only one that doesn't match Suyin's expectation (304 here instead of 308)
df_cpt1 = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Pfizer_mCRPC_CPT1.csv')
df_cpt2 = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Pfizer_mCRPC_CPT2.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,20,75,76,123,124,139,140,299,300,531,532) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
# Number of CPT predictors
prog = re.compile(r'^C')
colnames_cpt1 = df_cpt1.columns
set_prefixes_cpt1 = set([colname.split('_')[0] for colname in colnames_cpt1])
colnames_cpt2 = df_cpt2.columns
set_prefixes_cpt2 = set([colname.split('_')[0] for colname in colnames_cpt2])

cpt1 = [prog.match(prefix).string for prefix in set_prefixes_cpt1 if prog.match(prefix)]
cpt2 = [prog.match(prefix).string for prefix in set_prefixes_cpt2 if prog.match(prefix)]

In [85]:
# Only 304 CPT predictors
len(cpt2) + len(cpt1)

304

In [90]:
# Quick calculation with shape shows the same results
(df_cpt1.shape[1] - 1 + df_cpt2.shape[1] - 1)/4

304.0